# Stock Market Time Series Forecasting & Market Intelligence
## NIFTY 50 Index & Multi-Stock Analytics

This notebook provides an end-to-end financial analytics pipeline including:
- Multi-stock selection
- Technical indicators
- Multi-horizon forecasting (7/30/90 days)
- Volatility modeling using GARCH
- Market regime detection (Bull/Bear)
- Streamlit dashboard integration

## 1. Import Libraries

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from prophet import Prophet
from arch import arch_model

warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8")


## 2. Multi-Stock Selector & Data Loader

In [ ]:

DATA_DIR = "archive (2)"

def load_market_data(symbol="NIFTY50_all"):
    df = pd.read_csv(os.path.join(DATA_DIR, f"{symbol}.csv"))
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df.set_index('Date', inplace=True)
    return df

SYMBOL = "NIFTY50_all"  # Change to RELIANCE, TCS, HDFCBANK etc.
df = load_market_data(SYMBOL)
df.head()


## 3. Exploratory Data Analysis

In [ ]:

plt.figure(figsize=(14,5))
plt.plot(df['Close'], label='Close')
plt.title(f"{SYMBOL} Price Trend")
plt.legend()
plt.show()


## 4. Technical Indicators

In [ ]:

df['SMA_20'] = df['Close'].rolling(20).mean()
df['SMA_50'] = df['Close'].rolling(50).mean()
df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()

delta = df['Close'].diff()
gain = np.where(delta > 0, delta, 0)
loss = np.where(delta < 0, -delta, 0)
roll_up = pd.Series(gain).rolling(14).mean()
roll_down = pd.Series(loss).rolling(14).mean()
RS = roll_up / roll_down
df['RSI'] = 100 - (100 / (1 + RS))

df.dropna(inplace=True)
df.tail()


## 5. Multi-Horizon Forecasting (ARIMA)

In [ ]:

model = ARIMA(df['Close'], order=(5,1,0))
model_fit = model.fit()

forecast_7 = model_fit.forecast(7)
forecast_30 = model_fit.forecast(30)
forecast_90 = model_fit.forecast(90)

plt.figure(figsize=(14,6))
plt.plot(df['Close'], label='Historical')
plt.plot(forecast_7, label='7-Day')
plt.plot(forecast_30, label='30-Day')
plt.plot(forecast_90, label='90-Day')
plt.legend()
plt.title("Multi-Horizon Forecast")
plt.show()


## 6. Volatility Modeling with GARCH

In [ ]:

returns = 100 * df['Close'].pct_change().dropna()
garch = arch_model(returns, vol='Garch', p=1, q=1)
garch_fit = garch.fit(disp='off')

vol_forecast = garch_fit.forecast(horizon=30)
volatility = np.sqrt(vol_forecast.variance.values[-1])

plt.figure(figsize=(12,4))
plt.plot(volatility)
plt.title("Forecasted Volatility (30 Days)")
plt.show()


## 7. Market Regime Detection

In [ ]:

df['SMA_200'] = df['Close'].rolling(200).mean()
df['Regime'] = np.where(df['SMA_50'] > df['SMA_200'], 'Bull', 'Bear')

plt.figure(figsize=(14,6))
plt.plot(df['Close'], label='Close', alpha=0.7)
plt.scatter(df[df['Regime']=='Bull'].index, df[df['Regime']=='Bull']['Close'], 
            color='green', s=10, label='Bull')
plt.scatter(df[df['Regime']=='Bear'].index, df[df['Regime']=='Bear']['Close'], 
            color='red', s=10, label='Bear')
plt.legend()
plt.title("Market Regimes")
plt.show()


## 8. Streamlit Dashboard Code (Save as app.py)

In [ ]:

# Streamlit Dashboard (run separately)
# Save this cell as app.py

import streamlit as st

st.title("Market Forecasting Dashboard")

symbol = st.selectbox("Select Symbol", ["NIFTY50_all", "RELIANCE", "TCS", "HDFCBANK"])
df = load_market_data(symbol)

st.line_chart(df['Close'])

st.subheader("Market Regime")
st.line_chart(df[['SMA_50', 'SMA_200']])
